In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer

In [12]:
data = pd.read_csv("./goodbooks-10k/books_enriched.csv")
desc = data.description
desc = desc.dropna()
print(desc.info())
sentences = list(desc)

<class 'pandas.core.series.Series'>
Index: 9943 entries, 0 to 9999
Series name: description
Non-Null Count  Dtype 
--------------  ----- 
9943 non-null   object
dtypes: object(1)
memory usage: 155.4+ KB
None


In [13]:
model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = model.encode(sentences)
print(embeddings.shape)

(9943, 384)


In [14]:
similarities = model.similarity(embeddings, embeddings)

In [15]:
print(similarities.shape)
print(type(similarities))

torch.Size([9943, 9943])
<class 'torch.Tensor'>


In [43]:
def find_similar():
    for i in range(similarities.shape[0]):
        for j in range(similarities.shape[1]):
            if 0.80 < similarities[i][j] < 1 and i != j:
                print(similarities[i][j])
                return sentences[i], sentences[j]
            
sentence1, sentence2 = find_similar()

with open('similar.txt', 'w') as f:
    f.write("Original Title 1: " + str(data[data.description == sentence1].original_title) + '\n')
    f.write("Title 1: " + str(data[data.description == sentence1].title) + '\n\n')
    f.write(sentence1 + '\n\n')
    f.write("Original Title 2: " + str(data[data.description == sentence2].original_title) + '\n')
    f.write("Title 2: " + str(data[data.description == sentence2].title) + '\n\n')
    f.write(sentence2 + '\n') 

tensor(0.8100)


In [45]:
ratings = pd.read_csv("goodbooks-10k/ratings.csv")
print(ratings.shape)
duplicates = ratings[ratings.duplicated(subset=['user_id', 'book_id'], keep=False)]
print(ratings.shape)
print(duplicates.shape)

(5976479, 3)
(5976479, 3)
(0, 3)


In [61]:
books_unique = data.drop_duplicates('original_title', keep='first')
books_unique = books_unique.dropna(subset=['original_title', 'description'])
print(books_unique.shape)

sentences = list(books_unique.description)
embeddings = model.encode(sentences)
print(embeddings.shape)

(9230, 30)
(9230, 384)


In [90]:
similarities = model.similarity(embeddings, embeddings)

def write_similar(i, j, first=True):
    authors1 = set(books_unique.iloc[i].authors.split(',')) 
    authors2 = set(books_unique.iloc[j].authors.split(','))
    print(authors1, authors2, authors1.intersection(authors2))
    if len(authors1.intersection(authors2)) != 0:
        return False
    
    if first:
        mode = 'w'
    else:
        mode = 'a'
    with open('similar.txt', mode) as f:
        f.write("Original Title 1: " + str(books_unique.iloc[i].original_title) + '\n')
        f.write("Title 1: " + str(books_unique.iloc[i].title) + '\n\n')
        f.write("Authors 1: " + str(books_unique.iloc[i].authors) + '\n\n')
        f.write(sentences[i] + '\n\n')
        f.write("Original Title 2: " + str(books_unique.iloc[j].original_title) + '\n')
        f.write("Authors 2: " + str(books_unique.iloc[j].authors) + '\n')
        f.write("Title 2: " + str(books_unique.iloc[j].title) + '\n\n')
        f.write(sentences[j] + '\n')
        f.write("-----------------------------------------------\n\n")
    return True

def find_similar():
    similar = []
    for i in range(1000):
        for j in range(similarities.shape[1]):
            if 0.7 < similarities[i][j] < 1 and i != j:
                # print(similarities[i][j])
                # print(i, j)
                similar.append((i, j))
    return similar

similar = find_similar()
print(len(similar))
first = True
for i, j in similar:
    first = write_similar(i, j, first=first)

478
{"'Suzanne Collins'"} {"'Suzanne Collins'"} {"'Suzanne Collins'"}
{"'J.R.R. Tolkien'"} {" 'David Wenzel'", " 'Sean Deming'", "'Chuck Dixon'", " 'J.R.R. Tolkien'"} set()
{"'Dan Brown'"} {"'Dan Brown'"} {"'Dan Brown'"}
{"'Jane Austen'"} {"'P.D. James'"} set()
{"'Anne Frank'", " 'B.M. Mooyaart-Doubleday'", " 'Eleanor Roosevelt'"} {" 'Susan Massotty'", "'Anne Frank'"} {"'Anne Frank'"}
{"'Anne Frank'", " 'B.M. Mooyaart-Doubleday'", " 'Eleanor Roosevelt'"} {" 'Anna Quindlen'", "'Ruud van der Rol'", " 'Plym Peters'", " 'Tony   Langham'", " 'Anne Frank'", " 'Rian Verhoeven'"} set()
{"'Anne Frank'", " 'B.M. Mooyaart-Doubleday'", " 'Eleanor Roosevelt'"} {" 'Anna Quindlen'", "'Ruud van der Rol'", " 'Plym Peters'", " 'Tony   Langham'", " 'Anne Frank'", " 'Rian Verhoeven'"} set()
{" 'Reg Keeland'", "'Stieg Larsson'"} {" 'Reg Keeland'", "'Stieg Larsson'"} {" 'Reg Keeland'", "'Stieg Larsson'"}
{"'Suzanne Collins'"} {"'Suzanne Collins'"} {"'Suzanne Collins'"}
{"'Suzanne Collins'"} {"'Suzanne Colli

In [58]:
books_unique.loc[4065, ['title', 'original_title', 'description']]
print(len(sentences), books_unique.shape

9943 (9274, 30)


In [88]:
books_unique.authors = books_unique.authors.str.replace('[', '')
books_unique.authors = books_unique.authors.str.replace(']', '')
books_unique.authors.sample(20)

4885                           'Claudia Gray'
3840                         'Peggy Rathmann'
7361                          'John le Carré'
773                'Sophocles', 'J.E. Thomas'
9799                           'Alex Marwood'
9504    'Elizabeth Gaskell', 'Shirley Foster'
4399                           'Kim Harrison'
9168                    'Sonali Deraniyagala'
9251                           'Nora Roberts'
3104                            'A.G. Howard'
644                            'Stephen King'
6958                      'Mireille Guiliano'
7922                           'Nancy Farmer'
8746                            'Neil Gaiman'
7313                            'Umberto Eco'
3693                          'Philipp Meyer'
1440                              'Anne Rice'
914         'Vincent Bugliosi', 'Curt Gentry'
4215                        'Shane Claiborne'
5825                              'Anne Rice'
Name: authors, dtype: object